In [3]:
import pandas as pd
import numpy as np
import surprise
import os

## Load from here books_df

In [4]:
books_df = pd.read_csv('books_info.csv', index_col=0)

C:\Users\mperp\anaconda3\envs\bbva-env\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\mperp\anaconda3\envs\bbva-env\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


## Load from here user_df

In [5]:
import pandas as pd

In [6]:
user_df = pd.read_csv('ratings.csv', index_col=0)

In [7]:


rating_map = {"This user doesn't have any rating": 0,
              "did not like it": 1, 
              "it was ok": 2, 
              "liked it": 3, 
              "really liked it":4, 
              "it was amazing" : 5}

user_df['Rating'] = user_df['Rating'].map(rating_map)

In [8]:
user_df = user_df[user_df['Rating']!=0]

## Matrix Factorization (user_df)

In [9]:
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate
from surprise.model_selection.search import GridSearchCV
reader = Reader(rating_scale=(1, 5))

In [10]:
user_df

,ID,Name,Rating
0,1,Agile Web Development with Rails: A Pragmatic ...,5
1,1,The Restaurant at the End of the Universe (Hit...,5
2,1,Siddhartha,5
3,1,The Clock of the Long Now: Time and Responsibi...,4
4,1,"Ready Player One (Ready Player One, #1)",4
...,...,...,...
127673,10978,The Foundation: A Great American Secret: How P...,3
127674,10986,Cosette: The Sequel to Les Miserables,4
127675,10986,J. D. Salinger's The Catcher in the Rye,5
127676,10988,Facing the Lion: Growing Up Maasai on the Afri...,3


In [11]:
n_items = user_df['Name'].nunique()

In [12]:
user_df.columns = ['user_id', 'item_des' , 'ratings']
dataset_df = Dataset.load_from_df(user_df, reader)

In [26]:
# SVD 

factors = int(n_items**0.25+1)
print(factors)

grid = {'n_factors': [2,3,5,7,10,13,16,20,23], 'n_epochs': [300]}
gs = GridSearchCV(SVD, grid, cv=5, n_jobs=-1)

18


In [27]:
gs.fit(dataset_df)

KeyboardInterrupt: 

In [ ]:
# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

In [ ]:
gs.cv_results

In [17]:

#cross_validation = cross_validate(gs, dataset_df, measures=['RMSE'], cv=5, verbose=True)

## MF Author_id

In [18]:
books_df = pd.read_csv('books_info.csv', index_col=0)

In [19]:
books_df.columns

Index(['pagesNumber', 'Authors', 'Publisher', 'Rating', 'Language',
       'RatingDistTotal', 'RatingDist5', 'RatingDist3', 'CountsOfReview',
       'PublishDay', 'ISBN', 'RatingDist4', 'PublishMonth', 'Id',
       'PublishYear', 'RatingDist1', 'RatingDist2', 'Name', 'Description',
       'Count of text reviews', 'PagesNumber', 'page_number'],
      dtype='object')

In [20]:
books_df = books_df.drop_duplicates(subset='Name')

In [21]:
books_df.columns

Index(['pagesNumber', 'Authors', 'Publisher', 'Rating', 'Language',
       'RatingDistTotal', 'RatingDist5', 'RatingDist3', 'CountsOfReview',
       'PublishDay', 'ISBN', 'RatingDist4', 'PublishMonth', 'Id',
       'PublishYear', 'RatingDist1', 'RatingDist2', 'Name', 'Description',
       'Count of text reviews', 'PagesNumber', 'page_number'],
      dtype='object')

In [22]:
books_col_selected = ['Name', 'Authors']
b_df = books_df[books_col_selected]
b_df.columns = ['item_des', 'Authors']
user_books_df = user_df.merge(b_df, how='left', on='item_des')


In [23]:
user_books_df = user_books_df.fillna('Unknown')

In [24]:
user_books_df

,user_id,item_des,ratings,Authors
0,1,Agile Web Development with Rails: A Pragmatic ...,5,Dave Thomas
1,1,The Restaurant at the End of the Universe (Hit...,5,Douglas Adams
2,1,Siddhartha,5,Hermann Hesse
3,1,The Clock of the Long Now: Time and Responsibi...,4,Unknown
4,1,"Ready Player One (Ready Player One, #1)",4,Unknown
...,...,...,...,...
357826,10978,The Foundation: A Great American Secret: How P...,3,Joel L. Fleishman
357827,10986,Cosette: The Sequel to Les Miserables,4,Laura Kalpakian
357828,10986,J. D. Salinger's The Catcher in the Rye,5,Unknown
357829,10988,Facing the Lion: Growing Up Maasai on the Afri...,3,Joseph Lemasolai Lekuton


In [25]:
user_books_df.Authors.nunique()

23294

In [ ]:
algo_auth = SVD(n_factors=12, n_epochs=20, random_state=1 )
cross_validation = cross_validate(algo_auth, authors_dat, measures=['RMSE'], cv=5, verbose=True, n_jobs=-1)

## Mix User_df MF and Author 

We want to introduce bias for every user for an author. To do that I am going to compute this

1. Let us call A to the matrix with user_id, item_id and ratings matrix. A (Nusers x Nitems)
2. And A2, to the matrix with user_id, author_id and ratings matrix. This matrix will have the same dimension as A, there will be duplicates with user_id and author_id because we want to sum the same values. This is going to be computed with an Window function to maintain the same elements.
3. A' will be the weighted sum of A and A2. A' = k * A + (1 - k) * A2  for k in [0,1] 

A' will be the weights used for training, but the error will be computed with A.




In [34]:
from surprise.model_selection import KFold
from surprise import accuracy

In [ ]:
class SVD_mix(SVD):
    def __init__(self, n_factors=30, n_epochs=25, random_state=1, k=0.2, col_extra='col'):
        self.k = k
        self.col_extra = col_extra
        
    def fit(self, trainset):
        

In [51]:
reader_mix = Reader(rating_scale=(1, 5), line_format='user item rating', sep=' ')

In [52]:
user_books_df['ratings_author'] = user_books_df.groupby(['Authors', 'item_des'])['ratings'].transform(np.mean)

In [53]:
alpha = 0.2

user_books_df['rating_conj'] = (1 - alpha) * user_books_df['ratings'] + alpha*user_books_df['ratings_author']

In [54]:

user_authors_dat = Dataset.load_from_df(user_books_df[['user_id','item_des','rating_conj']], reader, )
user_authors_real = Dataset.load_from_df(user_books_df[['user_id','item_des','ratings']], reader, )

In [55]:
algo = SVD(n_factors=30, n_epochs=25, random_state=1)
kf = KFold(n_splits=5, random_state=1)

In [56]:
for uad, uar in zip(kf.split(user_authors_dat), kf.split(user_authors_real)):
    trainset, _ = uad
    _, testset = uar
    
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)


RMSE: 0.8852
RMSE: 0.8791
RMSE: 0.8772
RMSE: 0.8816
RMSE: 0.8780


In [57]:
uad_ex=kf.split(user_authors_dat)
uar_ex=kf.split(user_authors_real)

In [58]:
list(uad_ex)[0]

(<surprise.trainset.Trainset at 0x170f1076760>,
 [(5311, 'سرباز کوچک', 3.0),
  (1046, 'The Adventures of Huckleberry Finn', 3.978540772532189),
  (9609, 'Gone Girl', 3.1410334346504563),
  (4806, 'On the Day You Were Born', 3.1),
  (1597, 'Los límites de la Fundación (Fundación, #4)', 4.0),
  (4825, 'The Medium is the Massage', 3.0),
  (5250, 'Two Days (Private Club, #2)', 3.0),
  (2649,
   'The Tipping Point: How Little Things Can Make a Big Difference',
   3.9605839416058393),
  (3987, 'The Death Cure (The Maze Runner, #3)', 3.7333333333333334),
  (650, 'Madame Bovary', 3.1362962962962966),
  (1617,
   'The Man Who Mistook His Wife for a Hat and Other Clinical Tales',
   4.7862068965517235),
  (1910, 'The Valley of Fear', 2.9000000000000004),
  (7805, "Midnight's Children", 4.8330275229357795),
  (3239, 'To Kill a Mockingbird', 4.063614457831325),
  (10058, 'The Alchemist', 3.0776978417266188),
  (5028,
   "I'm a Stranger Here Myself: Notes on Returning to America After Twenty Years 

In [59]:
list(uar_ex)[1]

(<surprise.trainset.Trainset at 0x170eb1654f0>,
 [(2810, 'Seven Surrenders (Terra Ignota, #2)', 5.0),
  (1626, 'The Giver (The Giver, #1)', 5.0),
  (3961, 'Parasyte, Volume 4', 4.0),
  (151, 'The Client', 3.0),
  (7268, 'How to Be Popular', 3.0),
  (3769, 'The Story of Philosophy', 3.0),
  (678, 'The Darkest Kiss (Riley Jenson Guardian #6)', 3.0),
  (2597, 'Frankissstein: A Love Story', 3.0),
  (3628, 'The Book of Atrix Wolfe', 3.0),
  (3259, 'An Ice Cold Grave (Harper Connelly, #3)', 4.0),
  (6765, 'Before the Frost (Linda Wallander #1)', 4.0),
  (1597, 'The Catcher in the Rye', 3.0),
  (3854, 'Naked', 4.0),
  (5954, 'Harry Potter and the Deathly Hallows (Harry Potter, #7)', 4.0),
  (4390, 'His Christmas Bride', 1.0),
  (7092, "Children of the Mind (Ender's Saga, #4)", 3.0),
  (3132, 'The Gambler', 4.0),
  (1262, 'The Alice Network', 4.0),
  (696, 'A Changed Man', 2.0),
  (10492, 'A Feast for Crows (A Song of Ice and Fire, #4)', 2.0),
  (1416, 'Scythe (Arc of a Scythe, #1)', 4.0),
  (

## Softmax model

In [79]:

user_books_df_g

,ID,Name,Rating,page_number,PublishYear,Authors
0,1,[Agile Web Development with Rails: A Pragmatic...,"[5, 5, 5, 4, 4, 5, 5, 5, 5, 5, 5, 5, 3, 3, 5, ...","[558, 250, 144, Unknown, Unknown, Unknown, 192...","[2005, 1981, 2000, Unknown, Unknown, 2008, 195...","[Dave Thomas, Douglas Adams, Hermann Hesse, Un..."
1,2,"[The Time Traveler's Wife, My Sister's Keeper,...","[5, 4, 5, 3, 4, 4, 5, 5, 5, 5, 3, 4, 4, 5, 4, ...","[546, 712, 144, 620, Unknown, 458, Unknown, 97...","[2004, 2004, 1964, 2007, 2008, 1979, 2008, 201...","[Audrey Niffenegger, Jodi Picoult, Ian Fleming..."
2,3,"[The Ruby Programming Language, Pro CSS and HT...","[5, 3, 2, 3, 5, 4, 4, 5, 4, 3, 4, 3, 3, 3, 5, ...","[Unknown, 494, 280, Unknown, Unknown, Unknown,...","[2008, 2007, 2007, 2007, Unknown, Unknown, 200...","[David Flanagan, Michael Bowers, Kevin Mars..."
3,4,"[Jasmine Toguchi, Flamingo Keeper, Good Night,...","[4, 5, 5, 4, 4, 5]","[Unknown, 32, 32, 5, 221, Unknown]","[Unknown, 1996, 1996, 2000, 2005, Unknown]","[Unknown, Peggy Rathmann, Sandra Boynton, C.S...."
4,5,"[The Secret Garden, Remarkable Women of Califo...","[5, 5, 5, 4, 5, 3, 4, 5, 5, 5, 3, 5, 4, 5, 3, ...","[48, Unknown, 896, 384, Unknown, 609, 287, 288...","[1995, Unknown, 2002, 1978, Unknown, 2007, 200...","[Deborah Hautzig, Unknown, John Galsworthy, Ma..."
...,...,...,...,...,...,...
8914,10995,[Rating],[0],[Unknown],[Unknown],[Unknown]
8915,10996,"[The Catcher in the Rye, Harry Potter and the ...","[5, 5, 5, 5, 4, 5, 5, 5, 5, 4]","[220, 309, 766, 366, 607, 936, 448, 249, 272, ...","[1986, 1999, 2003, 2000, 2005, 2000, 2006, 200...","[J.D. Salinger, J.K. Rowling, J.K. Rowling, J...."
8916,10997,"[Animal Farm / 1984, بار هستی, The Little Prince]","[3, 4, 5]","[400, Unknown, 93]","[2003, Unknown, 2000]","[George Orwell, Unknown, Antoine de Saint-Exup..."
8917,10998,[Rating],[0],[Unknown],[Unknown],[Unknown]


In [89]:
user_books_df[user_books_df['ID']==10999]

,ID,Name,Rating,page_number,PublishYear,Authors
243112,10999,Rating,0,Unknown,Unknown,Unknown


## Google Collab functions

In [33]:
def pad(x, fill):
    return pd.DataFrame.from_dict(x).fillna(fill).values

def make_batch(ratings, batch_size):
    """Creates a batch of examples.
    Args:
    ratings: A DataFrame of ratings such that examples["movie_id"] is a list of
      movies rated by a user.
    batch_size: The batch size. 
    
    """    

    movie = ratings['Name_id'].values
    year = ratings['PublishYear_id'].values
    page_number = ratings['page_number_id'].values
    authors = ratings['Authors_id'].values
    label = ratings['Name_id'].values
    
    movie_l = []
    year_l = []
    page_number_l = []
    authors_l = []
    label_l = []
    for ms, ys, ps, auts, ls in zip(movie, year, page_number, authors, label):
        movie_l.append(ms)
        year_l.append(ys)
        page_number_l.append(ps)
        authors_l.append(auts)
        label_l.append([int(i) for i in ls])


    features = {
        "Name_id": pad(movie_l, ""),
        "PublishYear_id": pad(year_l, ""),
        "page_number_id": pad(page_number_l, ""),
        "Authors_id": pad(authors_l, ""),
        "label": pad(label_l, -1)
    
    }
    print(features)
    batch = (
      tf.data.Dataset.from_tensor_slices(features)
      .shuffle(1000)
      .repeat()
      .batch(batch_size)
      .make_one_shot_iterator()
      .get_next())
    return batch

    
def select_random(x):
    """Selectes a random elements from each row of x."""
    def to_float(x):
        return tf.cast(x, tf.float32)
    def to_int(x):
        return tf.cast(x, tf.int64)
    batch_size = tf.shape(x)[0]
    rn = tf.range(batch_size)
    nnz = to_float(tf.count_nonzero(x >= 0, axis=1))
    rnd = tf.random_uniform([batch_size])
    ids = tf.stack([to_int(rn), to_int(nnz * rnd)], axis=1)
    return to_int(tf.gather_nd(x, ids))

In [5]:
def split_dataframe(df, holdout_fraction=0.1):
    """Splits a DataFrame into training and test sets.
    Args:
    df: a dataframe.
    holdout_fraction: fraction of dataframe rows to use in the test set.
    Returns:
    train: dataframe for training
    test: dataframe for testing
    """
    test = df.sample(frac=holdout_fraction, replace=False)
    train = df[~df.index.isin(test.index)]
    return train, test

In [3]:
# @title Imports (run this cell)
from __future__ import print_function

import numpy as np
import pandas as pd
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.ERROR)

# Add some convenience functions to Pandas DataFrame.
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.3f}'.format
def mask(df, key, function):
    """Returns a filtered dataframe, by applying function to key"""
    return df[function(df[key])]

def flatten_cols(df):
    df.columns = [' '.join(col).strip() for col in df.columns.values]
    return df

Instructions for updating:
non-resource variables are not supported in the long term


In [7]:
def build_rating_sparse_tensor(ratings_df):
    """
    Args:
    ratings_df: a pd.DataFrame with `user_id`, `movie_id` and `rating` columns.
    Returns:
    A tf.SparseTensor representing the ratings matrix.
    """
    # ========================= Complete this section ============================
    indices = ratings_df[['ID', 'Name_id']].values
    values = ratings_df['Rating'].values
    # ============================================================================

    return tf.SparseTensor(
      indices=indices,
      values=values,
      dense_shape=[users.shape[0], movies.shape[0]])

In [8]:
user_books_df

NameError: name 'user_books_df' is not defined

In [9]:
def sparse_mean_square_error(sparse_ratings, user_embeddings, movie_embeddings):
    """
    Args:
    sparse_ratings: A SparseTensor rating matrix, of dense_shape [N, M]
    user_embeddings: A dense Tensor U of shape [N, k] where k is the embedding
      dimension, such that U_i is the embedding of user i.
    movie_embeddings: A dense Tensor V of shape [M, k] where k is the embedding
      dimension, such that V_j is the embedding of movie j.
    Returns:
    A scalar Tensor representing the MSE between the true ratings and the
      model's predictions.
    """
    # ========================= Complete this section ============================
    predictions = tf.gather_nd(
      tf.matmul(user_embeddings, movie_embeddings, transpose_b=True),
      sparse_ratings.indices)
    loss = tf.losses.mean_squared_error(sparse_ratings.values, predictions)
    # ============================================================================
    return loss

In [10]:
# @title CFModel helper class (run this cell)
class CFModel(object):
    """Simple class that represents a collaborative filtering model"""
    def __init__(self, embedding_vars, loss, metrics=None):
        """Initializes a CFModel.
        Args:
          embedding_vars: A dictionary of tf.Variables.
          loss: A float Tensor. The loss to optimize.
          metrics: optional list of dictionaries of Tensors. The metrics in each
            dictionary will be plotted in a separate figure during training.
        """
        self._embedding_vars = embedding_vars
        self._loss = loss
        self._metrics = metrics
        self._embeddings = {k: None for k in embedding_vars}
        self._session = None

    @property
    def embeddings(self):
        """The embeddings dictionary."""
        return self._embeddings

    def train(self, num_iterations=100, learning_rate=1.0, plot_results=True,
            optimizer=tf.train.GradientDescentOptimizer):
        """Trains the model.
        Args:
          iterations: number of iterations to run.
          learning_rate: optimizer learning rate.
          plot_results: whether to plot the results at the end of training.
          optimizer: the optimizer to use. Default to GradientDescentOptimizer.
        Returns:
          The metrics dictionary evaluated at the last iteration.
        """
        with self._loss.graph.as_default():
            opt = optimizer(learning_rate)
            train_op = opt.minimize(self._loss)
            local_init_op = tf.group(
                tf.variables_initializer(opt.variables()),
                tf.local_variables_initializer())
            if self._session is None:
                self._session = tf.Session()
            with self._session.as_default():
                self._session.run(tf.global_variables_initializer())
                self._session.run(tf.tables_initializer())
                tf.train.start_queue_runners()

        with self._session.as_default():
            local_init_op.run()
            iterations = []
            metrics = self._metrics or ({},)
            metrics_vals = [collections.defaultdict(list) for _ in self._metrics]

          # Train and append results.
            for i in range(num_iterations + 1):
                _, results = self._session.run((train_op, metrics))
            if (i % 10 == 0) or i == num_iterations:
                print("\r iteration %d: " % i + ", ".join(
                    ["%s=%f" % (k, v) for r in results for k, v in r.items()]),
                    end='')
                iterations.append(i)
                for metric_val, result in zip(metrics_vals, results):
                    for k, v in result.items():
                        metric_val[k].append(v)

            for k, v in self._embedding_vars.items():
                self._embeddings[k] = v.eval()

        if plot_results:
            # Plot the metrics.
            num_subplots = len(metrics)+1
            fig = plt.figure()
            fig.set_size_inches(num_subplots*10, 8)
            for i, metric_vals in enumerate(metrics_vals):
                ax = fig.add_subplot(1, num_subplots, i+1)
                for k, v in metric_vals.items():
                    ax.plot(iterations, v, label=k)
                ax.set_xlim([1, num_iterations])
                ax.legend()
        return results

In [11]:
def build_model(ratings, embedding_dim=3, init_stddev=1.):
    """
    Args:
    ratings: a DataFrame of the ratings
    embedding_dim: the dimension of the embedding vectors.
    init_stddev: float, the standard deviation of the random initial embeddings.
    Returns:
    model: a CFModel.
    """
    # Split the ratings DataFrame into train and test.
    train_ratings, test_ratings = split_dataframe(ratings)
    # SparseTensor representation of the train and test datasets.
    A_train = build_rating_sparse_tensor(train_ratings)
    A_test = build_rating_sparse_tensor(test_ratings)
    # ============================================================================
    # Initialize the embeddings using a normal distribution.
    U = tf.Variable(tf.random_normal(
      [A_train.dense_shape[0], embedding_dim], stddev=init_stddev))
    V = tf.Variable(tf.random_normal(
      [A_train.dense_shape[1], embedding_dim], stddev=init_stddev))
    # ========================= Complete this section ============================
    train_loss = sparse_mean_square_error(A_train, U, V)
    test_loss = sparse_mean_square_error(A_test, U, V)
    # ============================================================================
    metrics = {
      'train_error': train_loss,
      'test_error': test_loss
    }
    embeddings = {
      "user_id": U,
      "movie_id": V
    }
    return CFModel(embeddings, train_loss, [metrics])

### Softmax

In [12]:
def build_softmax_model(rated_movies, embedding_cols, hidden_dims):
    """Builds a Softmax model for MovieLens.
    Args:
    rated_movies: DataFrame of traing examples.
    embedding_cols: A dictionary mapping feature names (string) to embedding
      column objects. This will be used in tf.feature_column.input_layer() to
      create the input layer.
    hidden_dims: int list of the dimensions of the hidden layers.
    Returns:
    A CFModel object.
    """
    def create_network(features):
        """Maps input features dictionary to user embeddings.
        Args:
          features: A dictionary of input string tensors.
        Returns:
          outputs: A tensor of shape [batch_size, embedding_dim].
        """
        # Create a bag-of-words embedding for each sparse feature.
        inputs = tf.feature_column.input_layer(features, embedding_cols)
        # Hidden layers.
        input_dim = inputs.shape[1].value
        for i, output_dim in enumerate(hidden_dims):
            w = tf.get_variable(
                "hidden%d_w_" % i, shape=[input_dim, output_dim],
                initializer=tf.truncated_normal_initializer(
                  stddev=1./np.sqrt(output_dim))) / 10.
            outputs = tf.matmul(inputs, w)
            input_dim = output_dim
            inputs = outputs
        return outputs

    train_rated_movies, test_rated_movies = split_dataframe(rated_movies)
    train_batch = make_batch(train_rated_movies, 200)
    test_batch = make_batch(test_rated_movies, 100)

    with tf.variable_scope("model", reuse=False):
        # Train
        print(train_batch)
        train_user_embeddings = create_network(train_batch)
        train_labels = select_random(train_batch["label"])
    with tf.variable_scope("model", reuse=True):
        # Test
        test_user_embeddings = create_network(test_batch)
        test_labels = select_random(test_batch["label"])
        movie_embeddings = tf.get_variable(
            "input_layer/movie_id_embedding/embedding_weights")

    # ========================= Complete this section ============================
    test_loss = softmax_loss(
      test_user_embeddings, movie_embeddings, test_labels)
    train_loss = softmax_loss(
      train_user_embeddings, movie_embeddings, train_labels)
    _, test_precision_at_10 = tf.metrics.precision_at_k(
      labels=test_labels,
      predictions=tf.matmul(test_user_embeddings, movie_embeddings, transpose_b=True),
      k=10)
    # ============================================================================

    metrics = (
      {"train_loss": train_loss, "test_loss": test_loss},
      {"test_precision_at_10": test_precision_at_10}
    )
    embeddings = {"movie_id": movie_embeddings}
    return CFModel(embeddings, train_loss, metrics)

In [117]:
user_books_df = user_books_df.fillna('Unknown')
def _to_str_year(y):
    try:
        return str(int(y))
    except:
        return str(y)

user_books_df['page_number'] = user_books_df['page_number'].apply(lambda y: _to_str_year(y))
user_books_df['PublishYear'] = user_books_df['PublishYear'].apply(lambda y: _to_str_year(y))

In [69]:
user_books_df.to_csv('user_books_join.csv')

In [23]:
user_books_df = pd.read_csv('user_books_join.csv', index_col = 0)
user_books_df

,ID,Name,Rating,page_number,PublishYear,Authors
0,1,Agile Web Development with Rails: A Pragmatic ...,5,558,2005,Dave Thomas
1,1,The Restaurant at the End of the Universe (Hit...,5,250,1981,Douglas Adams
2,1,Siddhartha,5,144,2000,Hermann Hesse
3,1,The Clock of the Long Now: Time and Responsibi...,4,Unknown,Unknown,Unknown
4,1,"Ready Player One (Ready Player One, #1)",4,Unknown,Unknown,Unknown
...,...,...,...,...,...,...
362591,10978,The Foundation: A Great American Secret: How P...,3,357,2007,Joel L. Fleishman
362592,10986,Cosette: The Sequel to Les Miserables,4,652,1995,Laura Kalpakian
362593,10986,J. D. Salinger's The Catcher in the Rye,5,Unknown,Unknown,Unknown
362594,10988,Facing the Lion: Growing Up Maasai on the Afri...,3,128,2005,Joseph Lemasolai Lekuton


In [27]:
user_books_df = user_books_df[user_books_df['Rating']>0]


In [28]:
name_dict = {book: str(i) for i, book in enumerate(user_books_df.Name.unique())}
page_number_dict = {book: str(i) for i, book in enumerate(user_books_df.page_number.unique())}
year_dict = {book: str(i) for i, book in enumerate(user_books_df.PublishYear.unique())}
author_dict = {book: str(i) for i, book in enumerate(user_books_df.Authors.unique())}

In [29]:
user_books_df['Name_id'] = user_books_df['Name'].map(name_dict)
user_books_df['page_number_id'] = user_books_df['page_number'].map(page_number_dict)
user_books_df['PublishYear_id'] = user_books_df['PublishYear'].map(year_dict)
user_books_df['Authors_id'] = user_books_df['Authors'].map(author_dict)

<ipython-input-29-c1e37e8b602c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_books_df['Name_id'] = user_books_df['Name'].map(name_dict)
<ipython-input-29-c1e37e8b602c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_books_df['page_number_id'] = user_books_df['page_number'].map(page_number_dict)
<ipython-input-29-c1e37e8b602c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [30]:
user_books_df_g = (user_books_df
                .groupby("ID", as_index=False)
                .aggregate(lambda x: list(x)))


In [31]:
user_books_df_g

,ID,Name,Rating,page_number,PublishYear,Authors,Name_id,page_number_id,PublishYear_id,Authors_id
0,1,[Agile Web Development with Rails: A Pragmatic...,"[5, 5, 5, 4, 4, 5, 5, 5, 5, 5, 5, 5, 3, 3, 5, ...","[558, 250, 144, Unknown, Unknown, Unknown, 192...","[2005, 1981, 2000, Unknown, Unknown, 2008, 195...","[Dave Thomas, Douglas Adams, Hermann Hesse, Un...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 3, 3, 4, 5, 4, 3, 4, 3, 6, 7, 3, ...","[0, 1, 2, 3, 3, 4, 5, 6, 7, 3, 8, 9, 10, 11, 3...","[0, 1, 2, 3, 3, 4, 5, 6, 5, 3, 5, 7, 8, 9, 3, ..."
1,2,"[The Time Traveler's Wife, My Sister's Keeper,...","[5, 4, 5, 3, 4, 4, 5, 5, 5, 5, 3, 4, 4, 5, 4, ...","[546, 712, 144, 620, Unknown, 458, Unknown, 97...","[2004, 2004, 1964, 2007, 2008, 1979, 2008, 201...","[Audrey Niffenegger, Jodi Picoult, Ian Fleming...","[478, 479, 287, 480, 5, 481, 163, 58, 71, 482,...","[215, 216, 2, 217, 3, 23, 3, 37, 46, 218, 219,...","[16, 16, 26, 22, 4, 49, 4, 28, 0, 16, 18, 3, 3...","[218, 219, 18, 220, 4, 221, 18, 39, 50, 222, 2..."
2,3,"[The Ruby Programming Language, Pro CSS and HT...","[5, 3, 2, 3, 5, 4, 4, 5, 4, 3, 4, 3, 3, 3, 5, ...","[Unknown, 494, 280, Unknown, Unknown, Unknown,...","[2008, 2007, 2007, 2007, Unknown, Unknown, 200...","[David Flanagan, Michael Bowers, Kevin Mars...","[508, 509, 510, 511, 512, 513, 514, 515, 41, 5...","[3, 229, 52, 3, 3, 3, 105, 74, 24, 3, 230, 2, ...","[4, 22, 22, 22, 3, 3, 18, 18, 22, 4, 11, 25, 3...","[123, 237, 238, 239, 3, 3, 240, 241, 26, 242, ..."
3,4,"[Jasmine Toguchi, Flamingo Keeper, Good Night,...","[4, 5, 5, 4, 4, 5]","[Unknown, 32, 32, 5, 221, Unknown]","[Unknown, 1996, 1996, 2000, 2005, Unknown]","[Unknown, Peggy Rathmann, Sandra Boynton, C.S....","[547, 548, 250, 376, 549, 550]","[3, 120, 120, 9, 241, 3]","[3, 21, 21, 2, 0, 3]","[3, 264, 104, 33, 33, 3]"
4,5,"[The Secret Garden, Remarkable Women of Califo...","[5, 5, 5, 4, 5, 3, 4, 5, 5, 5, 3, 5, 4, 5, 3, ...","[48, Unknown, 896, 384, Unknown, 609, 287, 288...","[1995, Unknown, 2002, 1978, Unknown, 2007, 200...","[Deborah Hautzig, Unknown, John Galsworthy, Ma...","[435, 418, 551, 552, 553, 554, 555, 556, 557, ...","[69, 3, 242, 84, 3, 243, 143, 99, 244, 35, 106...","[14, 3, 25, 32, 3, 22, 18, 25, 14, 51, 20, 11,...","[207, 3, 265, 266, 3, 267, 151, 268, 269, 270,..."
...,...,...,...,...,...,...,...,...,...,...
4149,10986,"[The Kite Runner, 1984, The Sound and the Fury...","[4, 5, 5, 4, 2, 1, 3, 3, 3, 4, 5, 3, 4, 4, 4, ...","[329, 287, 368, 5, 309, 766, 607, 936, 448, 96...","[2007, 2006, 1966, 2007, 1999, 2003, 2005, 200...","[Khaled Hosseini, George Orwell, William Faulk...","[300, 270, 823, 499, 53, 112, 307, 389, 27, 13...","[158, 143, 174, 9, 33, 68, 161, 189, 15, 153, ...","[22, 18, 54, 22, 19, 12, 0, 2, 18, 16, 22, 12,...","[157, 143, 79, 13, 17, 17, 17, 17, 17, 683, 65..."
4150,10988,"[The Kite Runner, The Curious Incident of the ...","[5, 4, 4, 4, 4, 3, 4, 5, 4, 4, 5, 5, 3, 5, 3, 3]","[329, 272, 333, 561, 273, 63, 306, 252, 218, 4...","[2007, 2004, 2005, 2007, 1991, 2003, 2006, 200...","[Khaled Hosseini, Mark Haddon, Nick Hornby, Sa...","[300, 606, 4445, 1267, 2797, 919, 14220, 1166,...","[158, 41, 100, 381, 346, 327, 357, 180, 399, 6...","[22, 16, 0, 22, 35, 12, 18, 18, 39, 25, 31, 23...","[157, 309, 1196, 655, 1084, 489, 2730, 607, 59..."
4151,10993,"[Animal Farm, George Orwell's 1984: A Guide to...","[5, 5]","[96, Unknown]","[2004, Unknown]","[Ian Wooldridge, Unknown]","[1322, 37650]","[153, 3]","[16, 3]","[683, 3]"
4152,10996,"[The Catcher in the Rye, Harry Potter and the ...","[5, 5, 5, 5, 4, 5, 5, 5, 5, 4]","[220, 309, 766, 366, 607, 936, 448, 249, 272, ...","[1986, 1999, 2003, 2000, 2005, 2000, 2006, 200...","[J.D. Salinger, J.K. Rowling, J.K. Rowling, J....","[198, 53, 112, 441, 307, 389, 27, 10414, 10494...","[106, 33, 68, 204, 161, 189, 15, 39, 41, 107]","[42, 19, 12, 2, 0, 2, 18, 12, 15, 0]","[107, 17, 17, 17, 17, 17, 17, 3643, 314, 1203]"


In [ ]:
# Create feature embedding columns
def make_embedding_col(key, embedding_dim):
    categorical_col = tf.feature_column.categorical_column_with_vocabulary_list(
      key=key, vocabulary_list=list(set(user_books_df[key].values)), num_oov_buckets=0)
    return tf.feature_column.embedding_column(
      categorical_column=categorical_col, dimension=embedding_dim,
      # default initializer: trancated normal with stddev=1/sqrt(dimension)
      combiner='mean')



with tf.Graph().as_default():
      softmax_model = build_softmax_model(
          user_books_df_g,
          embedding_cols=[
              make_embedding_col("Name_id", 18),
              make_embedding_col("Authors_id", 12),
              make_embedding_col("PublishYear_id", 2),
              make_embedding_col("page_number_id", 2)
              ],
          hidden_dims=[18])

del user_books_df

{'Name_id': array([['0', '1', '2', ..., '', '', ''],
       ['478', '479', '287', ..., '', '', ''],
       ['508', '509', '510', ..., '', '', ''],
       ...,
       ['1322', '37650', '', ..., '', '', ''],
       ['198', '53', '112', ..., '', '', ''],
       ['7089', '36484', '316', ..., '', '', '']], dtype=object), 'PublishYear_id': array([['0', '1', '2', ..., '', '', ''],
       ['16', '16', '26', ..., '', '', ''],
       ['4', '22', '22', ..., '', '', ''],
       ...,
       ['16', '3', '', ..., '', '', ''],
       ['42', '19', '12', ..., '', '', ''],
       ['12', '3', '2', ..., '', '', '']], dtype=object), 'page_number_id': array([['0', '1', '2', ..., '', '', ''],
       ['215', '216', '2', ..., '', '', ''],
       ['3', '229', '52', ..., '', '', ''],
       ...,
       ['153', '3', '', ..., '', '', ''],
       ['106', '33', '68', ..., '', '', ''],
       ['185', '3', '163', ..., '', '', '']], dtype=object), 'Authors_id': array([['0', '1', '2', ..., '', '', ''],
       ['218', '21

In [ ]:
softmax_model.train(learning_rate=8., num_iterations=1000, optimizer=tf.train.AdagradOptimizer, verbose=True)

In [51]:
user_books_df

,ID,Name,Rating,page_number,PublishYear,Authors
0,1,Agile Web Development with Rails: A Pragmatic ...,5,558,2005,Dave Thomas
1,1,The Restaurant at the End of the Universe (Hit...,5,250,1981,Douglas Adams
2,1,Siddhartha,5,144,2000,Hermann Hesse
3,1,The Clock of the Long Now: Time and Responsibi...,4,Unknown,Unknown,Unknown
4,1,"Ready Player One (Ready Player One, #1)",4,Unknown,Unknown,Unknown
...,...,...,...,...,...,...
362591,10978,The Foundation: A Great American Secret: How P...,3,357,2007,Joel L. Fleishman
362592,10986,Cosette: The Sequel to Les Miserables,4,652,1995,Laura Kalpakian
362593,10986,J. D. Salinger's The Catcher in the Rye,5,Unknown,Unknown,Unknown
362594,10988,Facing the Lion: Growing Up Maasai on the Afri...,3,128,2005,Joseph Lemasolai Lekuton


In [ ]:
user_books_df